In [180]:
import numpy as np
import pandas as pd
import re
import warnings
import seaborn as sns
import matplotlib.pyplot as plt



In [181]:
df_sharks = pd.read_csv("./data/Shark_attacks.csv",encoding="unicode_escape")


In [182]:
df_sharks.shape
#df_sharks
#first we look at the shape and print a sample
df_sharks.sample(frac=0.4)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
24733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8337,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7264,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5486,1905.12.29,29-Dec-1905,1905.0,Invalid,AUSTRALIA,Western Australia,Geraldton,Bathing,Hugh Carroll,M,...,Shark involvement not confirmed,"The Advertiser, 12/30/1905",1905.12.29-Carroll.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1905.12.29,1905.12.29,817.0,NaN,NaN
18226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
812,2012.03.05,05-Mar-2012,2012.0,Unprovoked,REUNION,Saint-Benoit,Port de la Marine,Body boarding,Gerard Itema,M,...,NaN,"Clicanoo, 3/5/2012",2012.03.05-Itema.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2012.03.05,2012.03.05,5491.0,NaN,NaN


In [183]:
# check the empty columns and drop them, the 2 columns mentioned below seemed empty
df_sharks["Unnamed: 22"].isna().value_counts()
df_sharks["Unnamed: 23"].isna().value_counts()
df_sharks = df_sharks.drop(columns=["Unnamed: 22","Unnamed: 23"])


In [184]:
#check columns with irrelevent information and or not clear informations

df_sharks = df_sharks.drop(columns=["href formula","pdf","href","original order"])

In [185]:
#check for duplicates
#df_sharks[df_sharks.duplicated()]
df_sharks.duplicated().sum()


19418

In [186]:
#drop duplicates
df_sharks = df_sharks.drop_duplicates()

In [187]:
df_sharks.shape

(6305, 18)

In [188]:
#check and drop all the rows where EVERY value in that row is missing
df_sharks = df_sharks.dropna(axis = 0, how = 'all')
df_sharks.shape
# the dataset went from 25723 row to 6304 after deleting the empty ones

(6304, 18)

In [189]:
# the dataset went from 25723 row to 6304 after deleting the empty ones

In [190]:
#it's important to check columns name in order to avoid keyerror
print(df_sharks.columns)

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'Case Number.1', 'Case Number.2'],
      dtype='object')


In [191]:
#check if the 3 columns named "Case Number" are equal ==> they are not but it's because of mismatching format date
df_sharks["Case Number.1"].equals(df_sharks["Case Number.2"])
df_sharks["Case Number"].equals(df_sharks["Case Number.1"])
df_sharks["Case Number"].equals(df_sharks["Case Number.2"])

False

In [192]:
#it's not an information that I see useful, so I'll delete the columns, and also because I already have 2 other columns with dates
df_sharks = df_sharks.drop(columns=["Case Number.1","Case Number.2"])


In [193]:

df_sharks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF"
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com"
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com"
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF"
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper


In [194]:
#after deleting the the columns that are not relevant at this point, let's check the columns names again and clean them
print(df_sharks.columns)

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source'],
      dtype='object')


In [195]:
#printing the columns' names helps detect the hidden caracters that can't be detected from looking directly at the table.
#It seems that the colums "Sex" and "Species" have spaces at the end. lest's clean them

In [196]:
df_sharks.rename(columns={"Sex ":"Sex"}, inplace=True)
df_sharks.rename(columns={"Species ":"Species"}, inplace=True)

In [197]:

print(df_sharks.columns)

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source'],
      dtype='object')


In [198]:

df_sharks.shape

(6304, 16)

In [199]:
#making sure that all rows missing the columns in the code are dropped
df_sharks.dropna(axis=0, inplace=True, how="all", subset=["Age", "Type","Activity","Country","Fatal (Y/N)","Species"])

In [200]:
#now let's look at the content of each column and see if there are missing or mismatching info
#the 3 columns "Case Number,Date and Year" seem a little bit similar, so to make it more efficient and less redundant, 
#we'll split the Date column into 2 (month,year)
#and then drop the column Case Number, the Year column will be automatically replaced


In [201]:
df_sharks[["Month"]] = df_sharks["Date"].str.lower().str.extract(r'-(\w{3})-')
df_sharks[["Year"]] = df_sharks["Date"].str.extract(r'(\d{4})')



In [202]:
#now i'll drop the "Case Number" column, since i have a year and a month columns already
df_sharks = df_sharks.drop(columns=["Case Number"])



In [176]:
df_sharks.shape
df_sharks.head()
# now the dataset went from 8703 row and 24 columns to  (6304, 16) after deleting the empty ones

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Month
0,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",jun
1,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",jun
2,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",jun
3,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",jun
4,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,jun


In [178]:
#now we start cleaning the content, and check for empty fields
pd.isna(df_sharks).sum()

Date                         0
Year                        19
Type                         4
Country                     50
Area                       455
Location                   540
Activity                   544
Name                       210
Sex                        565
Age                       2831
Injury                      28
Fatal (Y/N)                539
Time                      3354
Species                   2838
Investigator or Source      17
Month                      910
dtype: int64

In [214]:
df_sharks['Type'].isna().value_counts()
#the "Type" column doesnt need much cleaning

False    6298
True        4
Name: Type, dtype: int64

In [215]:
df_sharks["Type"].unique()
#unify the data

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [216]:
df_sharks["Type"] = df_sharks["Type"].str.replace("Boating","Boat")
df_sharks["Type"] = df_sharks["Type"].str.replace("Boatomg","Boat")
df_sharks["Type"] = df_sharks["Type"].str.replace("Invalid","Unknown")

df_sharks["Type"].unique()


array(['Boat', 'Unprovoked', 'Unknown', 'Provoked', 'Questionable',
       'Sea Disaster', nan], dtype=object)

In [219]:
df_sharks["Country"].isna().sum()
#Replacing the empty fields in the Country column with "Unknown"


50

In [220]:
df_sharks["Country"].fillna("Unknown", inplace=True)
df_sharks["Country"].isna().sum()


In [ ]:
df_sharks["Area"].isna().sum()


In [222]:
df_sharks["Area"].fillna("Unknown", inplace=True)
df_sharks["Area"].isna().sum()

0

In [225]:
df_sharks["Location"].isna().sum()
df_sharks["Location"].fillna("Unknown", inplace=True)

In [224]:
#not much change in the columns "Area,Location,Country", except from filling all the blanks with "unknown"

In [228]:

df_sharks["Activity"].value_counts()

df_sharks["Activity"].isna().sum()


544

In [229]:
df_sharks["Name"].isna().sum()
df_sharks["Name"].fillna("Unknown", inplace=True)

In [230]:
#unify the column "Sex"
df_sharks["Sex"].unique()

array(['F', 'M', nan, 'M ', 'lli', 'N', '.'], dtype=object)

In [232]:
df_sharks["Sex"] = df_sharks["Sex"].str.replace("N", "M")
df_sharks["Sex"] = df_sharks["Sex"].str.replace("M ", "M")
df_sharks["Sex"] = df_sharks["Sex"].str.replace("lli", "Unknown")
df_sharks["Sex"] = df_sharks["Sex"].str.replace(".", "Unknown")
df_sharks["Sex"] = df_sharks["Sex"].str.replace("nan", "Unknown")
df_sharks['Sex'] = df_sharks['Sex'].fillna('Unknown')
df_sharks["Sex"].unique()

C:\Users\34603\AppData\Local\Temp\ipykernel_15524\3394812697.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_sharks["Sex"] = df_sharks["Sex"].str.replace(".", "Unknown")


array(['F', 'M', 'Unknown'], dtype=object)

In [234]:

df_sharks.sample(frac=0.5)



,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Month
358,20-Sep-2015,2015,Unprovoked,USA,Hawaii,"Upolu Point, North Kohala, Big Island",Spearfishing,Braxton Rocha,M,27,Severe laceration to left leg,N,15h52,"Tiger shark, 13'","Big Island Video, 9/20/2015",sep
350,07-Oct-2015,2015,Unprovoked,AUSTRALIA,Western Australia,Pyramids Beach,Surfing,Eli Zawadzki,M,18,Foot injured,N,16h50,NaN,"Daily Mail, 10/7/2015",oct
4613,1946,1946,Boat,SOUTH AFRICA,Western Cape Province,Table Bay,NaN,boat,Unknown,NaN,"Shark holed and sank boat, occupants rescued",N,NaN,NaN,"T. Wallett, p.27",NaN
5673,Reported 18-Jan-1893,1893,Unprovoked,FRENCH POLYNESIA,NaN,One week out of Pápete,Murdered,sailor,M,NaN,FATAL.Tossed overboard to sharks by Rodrigue b...,Y,NaN,NaN,"West Australian, 1/18/1893",jan
1732,05-Oct-2003,2003,Unprovoked,USA,Florida,"Ocean Reef Park, Singer Island, Palm Beach County",Surfing,female,F,NaN,Minor puncture wounds in hand,N,15h00,NaN,"J. Eager, scubaradio.com",oct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2701,1989,1989,Unprovoked,PAPUA NEW GUINEA,New Ireland,Kavieng,"Scuba diving, hand feeding sharks",Dinah Halstead,F,NaN,Thigh & calf bitten,N,NaN,7' silvertip shark,"S. Waterman, GSAF",NaN
1032,19-Apr-2010,2010,Unprovoked,USA,Hawaii,"Hanalei Bay, Kauai",Surfing,Jim Rawlinson,M,68,"No injury, surfboard bitten",N,16h00,14' shark,"R. Collier, Honolulu Advertiser, 4/23/2010",apr
5357,12-July-1913,1913,Unprovoked,USA,South Carolina,Charleston,NaN,soldier,M,NaN,FATAL,Y,NaN,NaN,"C. Creswell, GSAF",NaN
3038,Mar-1981,1981,Unprovoked,BRAZIL,Rio de Janeiro,Cabo Frio,Diving,NaN,Unknown,NaN,NaN,UNKNOWN,NaN,White shark,"Globo,",NaN


In [235]:
df_sharks["Age"].isna().sum()


2831

In [237]:
#df_sharks["Age"] ==> is not integer
print(df_sharks.dtypes)

Date                      object
Year                      object
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
Month                     object
dtype: object


In [241]:
df_sharks['Age'] = df_sharks['Age'].str.replace(r'[^\d.]+', '')
df_sharks['Age'] = df_sharks['Age'].str.replace('.', '')
df_sharks['Age'] = df_sharks['Age'].replace(np.nan, 0)
df_sharks['Age'] = df_sharks['Age'].replace('', "0")


C:\Users\34603\AppData\Local\Temp\ipykernel_15524\2135340958.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_sharks['Age'] = df_sharks['Age'].str.replace(r'[^\d.]+', '')
C:\Users\34603\AppData\Local\Temp\ipykernel_15524\2135340958.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_sharks['Age'] = df_sharks['Age'].str.replace('.', '')


In [243]:
df_sharks['Age'] = df_sharks['Age'].astype('int64')

In [245]:
df_sharks["Age"] = df_sharks["Age"].apply(lambda x: x if x < 99 else 0)

In [247]:
meanAge = df_sharks[df_sharks['Age'] > 0].mean()
meanAge

C:\Users\34603\AppData\Local\Temp\ipykernel_15524\1101999508.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  meanAge = df_sharks[df_sharks['Age'] > 0].mean()


Age    27.40047
dtype: float64

In [248]:

df_sharks['Age'] = df_sharks['Age'].apply(lambda x: x if x > 0 else meanAge)


In [249]:
df_sharks["Age"].mean()

27.400469759247567

In [49]:
df_cleaning["Fatal (Y/N)"].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [50]:
df_cleaning["Fatal (Y/N)"] = df_cleaning["Fatal (Y/N)"].str.replace(" N", "N")
df_cleaning["Fatal (Y/N)"] = df_cleaning["Fatal (Y/N)"].str.replace("N ", "N")
df_cleaning["Fatal (Y/N)"] = df_cleaning["Fatal (Y/N)"].str.replace("y", "Y")
df_cleaning["Fatal (Y/N)"] = df_cleaning["Fatal (Y/N)"].str.replace("2017", "UNKNOWN")
df_cleaning["Fatal (Y/N)"] = df_cleaning["Fatal (Y/N)"].str.replace("Unknown", "UNKNOWN")
df_cleaning["Fatal (Y/N)"] = df_cleaning["Fatal (Y/N)"].str.replace("M", "UNKNOWN")

df_cleaning["Fatal (Y/N)"] = df_cleaning["Fatal (Y/N)"].fillna("UNKNOWN")





In [51]:
df_cleaning["Fatal (Y/N)"].value_counts()


N          4301
Y          1389
UNKNOWN     614
Name: Fatal (Y/N), dtype: int64

In [52]:
df_cleaning.sample()

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Month
2718,22-Aug-1988,1988,Unprovoked,ITALY,Manfredonia,Ippocampo,NaN,male,M,16,Survived,N,NaN,NaN,"C. Moore, GSAF",aug


In [53]:
print(df_cleaning.columns)

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species',
       'Investigator or Source', 'Month'],
      dtype='object')


In [54]:
df_cleaning["Injury"].value_counts()


FATAL                                                                                     802
Survived                                                                                   97
Foot bitten                                                                                87
No injury                                                                                  82
Leg bitten                                                                                 72
                                                                                         ... 
Cut foot, but injury caused by fishing line, not the shark                                  1
Left foot bitten after he accidentally stepped on the shark         PROVOKED INCIDENT       1
2 puncture wounds in left leg                                                               1
PROVOKED INCIDENT    Knee bitten by shark trapped in net                                    1
FATAL. "Shark bit him in half, carrying away the lower extre

In [189]:
#sns.boxplot(x=df_cleaning["Country"], hue=df_cleaning["Fatal (Y/N)"], palette="magma")

In [1]:
#df_cleaning["Country"].value_counts().plot(kind="bar", color="salmon", title="Number of penguins by species")
#plt.xticks(rotation=0)


In [ ]:
for i in 

In [ ]:
#age/fatality
#sex/fatality
#activity/fatality